<a href="https://colab.research.google.com/github/emmanuelvaie/google_colab/blob/main/proto_dame_de_nages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get references data
Find Mattieu's file and read it into a dataframe


In [9]:
# piece of code that get a token
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate the user
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Once authenticated, list the files that contains personal records for the atheletes


In [32]:
# Auto-iterate through all files that matches this query
#file_list = drive.ListFile({'q': "'root' in parents and title contains 'Champs' and trashed=false"}).GetList()
#sharedWithMe=true 
# id = 1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q
file_list = drive.ListFile({'q': "trashed=false and title contains '-Nico - ' and title contains '2022'"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: 2022.10.08-Nico - 1.csv, id: 1Lo_LeTSpsU1XvKA26-sXJU03MsYklGIu
title: 2022.05.21-Nico - 5.csv, id: 1sAdUcDz4vxaMGMsCgcWc2i2uC6jVA5dv
title: 2022.05.15-Nico - 3.csv, id: 1a-OP8ilFEVL_bCioayaGJGDi1fh7M-Ol
title: 2022.05.15-Nico - 2.csv, id: 12kZ98a6IHGCWbQfguBA0Cb56CL2-REHL
title: 2022.05.15-Nico - 1.csv, id: 1E0nwCTgYgxOqU-tXJ3MludLZe4zF4f3Q


In [36]:

# we have the list
ddd_list = list()
lines2skip = {}

for ddd_file in file_list:
  file_id = ddd_file['id']
  # we are going to create an temp file in GDrive which is file.csv
  # this assume pyDrive has a token to read the GoogleDrive
  myfile = drive.CreateFile({'id': file_id})
  myfile.GetContentFile('file.csv')
  file1 = open('file.csv', 'r')
  count = 0

  for line in file1:
    count += 1
    #print("Line{}: {}".format(count, line.strip()))
    if line.startswith('Interval Summaries:') and count==19:
      #print('Line to skip is 28')
      lines2skip[file_id] = 28   
      break  
    else:
      if count==19:
        #print('Line to skip is 18')
        lines2skip[file_id] = 18     
        
# Closing files
file1.close()
lines2skip

{'1Lo_LeTSpsU1XvKA26-sXJU03MsYklGIu': 28,
 '1sAdUcDz4vxaMGMsCgcWc2i2uC6jVA5dv': 18,
 '1a-OP8ilFEVL_bCioayaGJGDi1fh7M-Ol': 18,
 '12kZ98a6IHGCWbQfguBA0Cb56CL2-REHL': 18,
 '1E0nwCTgYgxOqU-tXJ3MludLZe4zF4f3Q': 18}

once the id of the spreadsheet retrieved, load it. 
But first, we need to be allowed Gspread  / GSheet to access to our data.

In [37]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Get the data from the outriggers

In [38]:
import pandas as pd

# we have the list
ddd_list = list()
for ddd_file in file_list:
  # collect meta data from the filename
  filename = ddd_file['title']
  file_id = ddd_file['id']
  tokens = filename.split('-')
  rower = tokens[1].strip().lower()
  date = tokens[0].split('.')[2].strip() + '/' + tokens[0].split('.')[1].strip() + '/' + tokens[0].split('.')[0].strip()
  training_type = 'tour-'+ tokens[2].strip() 
  print('rower: %s, training type: %s, date: %s' % (rower, training_type, date))
  
  # this assume pyDrive has a token to read the GoogleDrive
  myfile = drive.CreateFile({'id': file_id})
  myfile.GetContentFile('file.csv')
  
  # then read it in a Dataframe
  skiprows = lines2skip[file_id]
  ddd = pd.read_csv('file.csv', skiprows=skiprows, header=None)

  # clean and organize the data
  # column names un 1st row
  # reformat
  headers = ddd.iloc[0].apply(lambda x: str(x).replace(' ', '_'))
  headers = headers.apply(lambda x: str(x).replace('(', ''))
  headers = headers.apply(lambda x: str(x).replace(')', ''))
  headers = headers.apply(lambda x: str(x).replace('.', ''))
  headers = headers.apply(lambda x: str(x).lower())

  # rename columns
  ddd = ddd.rename(columns=headers)

  # dropping NA values
  ddd2 = ddd.dropna()

  # drop the 2 first lines
  ddd2 = ddd2.drop([0, 1])

  # drop the 2 last rows
  #ddd2 = ddd2.drop(ddd2.tail(2).index)

  # reset the index
  ddd2 = ddd2.reset_index(drop=True)

  # changing column types
  column_names = list(ddd2.columns)

  # datetime columns
  # remove lines with date time columns formated to '---'
  ddd2 = ddd2.drop(ddd2[ddd2['elapsed_time'].map(lambda x: str(x) == '---')].index)
  ddd2 = ddd2.drop(ddd2[ddd2['split_gps'].map(lambda x: str(x) == '---')].index)
  ddd2 = ddd2.drop(ddd2[ddd2['split_imp'].map(lambda x: str(x) == '---')].index)

  # iterating on columns and change the types to string
  for col in column_names:
      ddd2[col] = ddd2[col].apply(lambda x: str(x).replace('---', '0'))

  ddd2 = ddd2.convert_dtypes()

  # removing datetime columns from the columns list
  column_names.pop(3)  # 'elapsed_time'
  column_names.pop(3)  # 'split_gps'
  column_names.pop(4)  # 'split_imp'

  # add meta data
  ddd2['rower'] = rower
  ddd2['training_type'] = training_type
  ddd2['date'] = date

  for col in column_names:
    ddd2[col] = pd.to_numeric(ddd2[col])

  # add columns amplitude and amplitude_efficace
  ddd2['amplitude'] = -1*(ddd2['catch'] - ddd2['finish'])
  ddd2['amplitude_efficace'] = - ddd2['catch'] - ddd2['slip'] + ddd2['finish'] - ddd2['wash']

  # add loc in google format
  ddd2['loc'] = ddd2['gps_lat'].apply(lambda x: str(x)) + ',' + ddd2['gps_lon'].apply(lambda x: str(x))

 
  # accumulate ddd
  ddd_list.append(ddd2)

# concat dataframes
all_ddd = pd.concat(ddd_list)


rower: nico, training type: tour-1.csv, date: 08/10/2022
rower: nico, training type: tour-5.csv, date: 21/05/2022
rower: nico, training type: tour-3.csv, date: 15/05/2022
rower: nico, training type: tour-2.csv, date: 15/05/2022
rower: nico, training type: tour-1.csv, date: 15/05/2022


In [39]:
all_ddd.count()

interval               1563
distance_gps           1563
distance_imp           1563
elapsed_time           1563
split_gps              1563
speed_gps              1563
split_imp              1563
speed_imp              1563
stroke_rate            1563
total_strokes          1563
distance/stroke_gps    1563
distance/stroke_imp    1563
heart_rate             1563
power                  1563
catch                  1563
slip                   1563
finish                 1563
wash                   1563
force_avg              1563
work                   1563
force_max              1563
max_force_angle        1563
gps_lat                1563
gps_lon                1563
rower                  1563
training_type          1563
date                   1563
amplitude              1563
amplitude_efficace     1563
loc                    1563
dtype: int64

In [40]:
all_ddd.dtypes

interval                 int64
distance_gps           float64
distance_imp           float64
elapsed_time            string
split_gps               string
speed_gps              float64
split_imp               string
speed_imp              float64
stroke_rate            float64
total_strokes            int64
distance/stroke_gps    float64
distance/stroke_imp    float64
heart_rate               int64
power                    int64
catch                    int64
slip                     int64
finish                   int64
wash                     int64
force_avg                int64
work                     int64
force_max                int64
max_force_angle          int64
gps_lat                float64
gps_lon                float64
rower                   object
training_type           object
date                    object
amplitude                int64
amplitude_efficace       int64
loc                     object
dtype: object

Adding a workseet with the dame de nage data

In [41]:
import gspread
import gspread_dataframe as gd

db_sh = gc.open_by_key('1u4KNFewcNQ7YpfTW0L1S2RM6Wu87xar-5zFzvXa2T2o')

worksheet_data = db_sh.worksheet('database')
gd.set_with_dataframe(worksheet_data,all_ddd, include_index=False,include_column_header=True,resize=True)